## Проверка номерных знаков на основе нейронной сети
В данном модуле происходит процесс валидации гос.номеров автомобилей при помощи простейшей нейронной сети, обучаемой распределением весов и коэфициентов смещений вручную, извлеченных через общий модуль plates_parser.py. Валидация требуется так, как извлечение номерных знаков при помощи регулярного выражения в данном источнике не дает однозначных результатов, по причине присутствия в строках других данных, по структуре подходящих для отбора регулярным выражением.

### Архитектура нейронной сети
Входной слой содержит 4 нейрона, представляющих различные свойства номерного знака. Нейронная сеть имеет один скрытый слой с 4 нейронами и выходной слой с 1 нейроном.

### Критерии валидации номерных знаков
Четыре нейрона входного слоя соответствуют четырем свойствам валидации номера:

1. Длина номера без пробелов 9 знаков
2. В номере присутствуют 3 буквы
3. В номере присутствуют 6 цифр
4. В номерных знаках присутствуют только те буквы кириллицы, которые имеют графическое сходство с буквами из латинского алфавита. Создается список допустимых букв, включающий в себя как заглавные, так и строчные буквы.

Сигмоидная функция активации: в данной нейронной сети используется сигмоидная функция активации, обеспечивающая нелинейность в приведении чисел к бинарному виду.

Цикл обучения: Мы задаем скорость обучения и количество эпох обучения. Эти параметры влияют на то, как сеть учится валидировать номерные знаки.

### Процесс валидации
Мы реализуем функцию is_valid_plate(plate), которая принимает строку номерного знака в качестве входа и выполняет валидацию с использованием нашей нейронной сети.

Процесс валидации включает следующие шаги:

Векторизация свойств номерного знака: Мы преобразуем свойства номерного знака в вектор, который служит входными данными для нашей нейронной сети. Мы извлекаем различные свойства, такие как длина номерного знака, количество букв, количество цифр и допустимость букв.

Прямое распространение: Мы выполняем прямое распространение через нашу нейронную сеть. Входной вектор умножается на ручно инициализированные веса и добавляется к смещениям для первого скрытого слоя. К выходу скрытого слоя применяется сигмоидная функция активации, и процесс повторяется для выходного слоя.

Интерпретация выхода: Выход сети представляет собой значение от 0 до 1, представляющее уверенность сети в том, что номерной знак действителен. Если выход близок к 1 (больше или равен 0,8), номерной знак считается действительным.

### Окончательная валидация
Мы используем функцию is_valid_plate(plate) для проверки действительности данного номерного знака. Результат показывает, действителен ли знак или нет.

In [3]:
import os
import sys

# Определяем путь к текущей папке
current_dir = os.getcwd()

# Определяем путь к корневой папке проекта, поднимаясь из текущей папке на один уровень
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

from __init__ import *
from DC_settings import *

In [4]:
from PROCESSOR.src1_api_get import main as src1_api_get
from PROCESSOR.src1_api_parse import main as src1_api_parse
from PROCESSOR.src_xlsx_get import main as src_xlsx_get
from PROCESSOR.src2_ops_parse import main as src2_ops_parse
from PROCESSOR.src3_drvs_parse import main as src3_drvs_parse
from PROCESSOR.src3_drvs_validator import main as src3_drvs_validator
from PROCESSOR.uniplates import main as uniplates
from PROCESSOR.user_report import main as user_report

In [5]:
src1_get = src1_api_get()
src1_parse = src1_api_parse(src1_get)
src2_get = src_xlsx_get(parent_dir, fname = 'dispatch')
src2_parse = src2_ops_parse(src2_get)
src3_get = src_xlsx_get(parent_dir, fname = 'drivers')
src3_parse = src3_drvs_parse(src3_get)

Link is accessible. Response: 200.


In [6]:
# Присваиваем входящий датафрейм новой переменной
df = src3_parse
pprint(df, set_pandas_options(df, width=1000, colwidth=20, colmap = False))

           Brands           Locations                Units               Plates              Drivers Ratings
0       Chevrolet      Центральный АО     Chevrolet Camaro          [О123ЕХ777]  Петров Александр...     4.5
1       Chevrolet      Центральный АО     Chevrolet Impala          [О456ЕР199]  Смирнова Елена Н...     4.2
2       Chevrolet      Центральный АО     Chevrolet Malibu  [С789НХ797,  Н77...  Кузнецов Сергей ...     4.8
3       Chevrolet      Центральный АО      Chevrolet Tahoe  [К321РО177,  Р12...  Васильева Анна А...     3.9
4       Chevrolet      Центральный АО    Chevrolet Equinox          [У654ОУ799]  Павлова Ольга Ми...     4.6
5       Chevrolet      Центральный АО   Chevrolet Traverse  [О987РЕ797,  Е23...  Михайлов Игорь В...     4.3
6       Chevrolet      Центральный АО  Chevrolet Silverado  [Р678СО120Н22,  ...  Иванов Ирина Сер...     4.9
7            Ford         Северный АО         Ford Mustang  [С361СС920У33,  ...  Иванов Александр...     4.8
8            Ford  

In [7]:
# Разбиваем колонку, состоящую из листов парных значений на 2 колонки, и загоняем их в датафрейм
L_plates = df['Plates'].tolist()
col_1, col_2 = [], []

# Итерируемся по списку парных значений и заполняем новые колонки
for i in L_plates:
    if len(i) > 1:
        col_1.append(i[0])
        col_2.append(i[1])
    else:
        col_1.append(i[0])
        col_2.append(np.nan)

# Удаляем пробелы из значений col_2 с помощью регулярных выражений
col_2 = [re.sub('\s*', '', str(x)) for x in col_2]
df_plates = pd.DataFrame(zip(col_1, col_2), columns = ['col_1', 'col_2'])
pprint(df_plates)

           col_1         col_2
0      О123ЕХ777           nan
1      О456ЕР199           nan
2      С789НХ797  Н777ОХ220Р44
3      К321РО177  Р123ОУ100К22
4      У654ОУ799           nan
5      О987РЕ797  Е234КС650Х55
6   Р678СО120Н22     Е456СР799
7   С361СС920У33     Р123РХ799
8      О197СС777  Х359РС120У33
9      У799СС799           nan
10     Р199ЕХ197  Н359ЕС120У33
11     У977ЕС199  Р359УС120У33
12     О123УХ777  С359СУ120Р33
13     Р799СЕ799  О199ОС120Е33
14     У797УХ777  К797РХ120С33
15  У797АА120С33     О777ОР799
16     Е799СС797  С199ЕС120У33
17     Р197ОС199  Х799РО120Е33
18     К199НС777  Е797РР120О33
19     О123УХ799  С359СС120У33
20     С456НУ799  Е111НС120О33
21     О789РО799           nan
22  О987УС120У33     С234ОР799
23     У512ОУ777  У799СС120У33
24     К888ЕХ799  У123ОС120У33
25     У798ОН799  Е797СО120О33
26     К111УО799           nan
27  С789ЕС120Е33     О123ЕР799
28     Р234ОС799  Е987СС120У33
29     У512УУ799           nan


In [8]:
# Определение функции нейросети
def neural_network(plate):
    # Определяем список допустимых буквенных значений
    legal_letters = ['А', 'В', 'Е', 'К', 'М', 'Н', 'О', 'Р', 'С', 'Т', 'У', 'Х', 'а', 'в', 'е', 'к', 'м', 'н', 'о', 'р', 'с', 'т', 'у', 'х']

    # Определяем архитектуру нейросети
    input_dim = 4
    hidden_dim = 4
    output_dim = 1

    # Назначаем веса и коэфициенты смещения для первого скрытого слоя вручную
    weights1 = np.array([[1, 0.6, 0.7, 0.8],
                        [0.2, 0.3, 0.4, 0.5],
                        [0.3, 0.4, 0.5, 0.6],
                        [0.4, 0.5, 0.6, 0.7]])

    bias1 = np.array([-0.4, -0.3, -0.6, -0.5])

    # Определяем функцию активации нейронов в виде сигмоидной функции
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Определям параметры для цикла обучения
    learning_rate = 0.1
    epochs = 1000

    # Определям функцию валидации гос.номеров при помощи нейросети
    def is_valid_plate(plate):
        # Лист для векторизации свойств гос.номеров
        input_data = []

        # Длина гос.номера
        input_data.append(1 if len(plate) == 9 and ' ' not in plate else 0)

        # Наличие 3 букв в строке
        input_data.append(2 if len([c for c in plate if c.isalpha()]) == 3 else 0)

        # Наличие 6 цифр в строке
        input_data.append(3 if len([c for c in plate if c.isdigit()]) == 6 else 0)

        # Буквы входят в список допустимых буквенных значений
        input_data.append(4 if all(c in legal_letters for c in plate if c.isalpha()) else 0)

        # Прямое распространение
        hidden_layer1 = np.dot(input_data, weights1) + bias1
        hidden_output1 = sigmoid(hidden_layer1)
        output_layer = np.dot(hidden_output1, weights1) + bias1
        output = sigmoid(output_layer)

        # Порог валидности 
        return output >= 0.8
    
    # Выполняем валидацию номерных знаков с использованием нейросети
    is_valid = is_valid_plate(plate)
    return is_valid

In [9]:
# Определяем списки для хранения результатов валидации
valid_1, valid_2 = [], []

# Итерируемся по датафрейму и выполняем валидацию с использованием нейросети
for index, (i, j) in enumerate(zip(df_plates['col_1'], df_plates['col_2'])):
    is_valid_1 = neural_network(i)
    valid_1.append(is_valid_1)
    is_valid_2 = neural_network(j)
    valid_2.append(is_valid_2)

# Создаем датафрейм с результатами валидации     
df_verif = pd.DataFrame(zip(col_1, valid_1, col_2, valid_2), columns = ['Plates_1', 'Verif_1', 'Plates_2', 'Verif_2'])
pprint(df_verif, set_pandas_options(df_verif, width=1000, colwidth=30, colmap = False))


        Plates_1                      Verif_1      Plates_2                       Verif_2
0      О123ЕХ777     [True, True, True, True]           nan  [False, False, False, False]
1      О456ЕР199     [True, True, True, True]           nan  [False, False, False, False]
2      С789НХ797     [True, True, True, True]  Н777ОХ220Р44   [False, False, False, True]
3      К321РО177     [True, True, True, True]  Р123ОУ100К22   [False, False, False, True]
4      У654ОУ799     [True, True, True, True]           nan  [False, False, False, False]
5      О987РЕ797     [True, True, True, True]  Е234КС650Х55   [False, False, False, True]
6   Р678СО120Н22  [False, False, False, True]     Е456СР799      [True, True, True, True]
7   С361СС920У33  [False, False, False, True]     Р123РХ799      [True, True, True, True]
8      О197СС777     [True, True, True, True]  Х359РС120У33   [False, False, False, True]
9      У799СС799     [True, True, True, True]           nan  [False, False, False, False]
10     Р19

In [10]:
# Обновлям датафрейм нулевыми значениями если гос.номер не прошел проверку на валидность
for index, (p1, v1, p2, v2) in df_verif[['Plates_1', 'Verif_1', 'Plates_2', 'Verif_2']].iterrows():
    if np.any(v1 == False):
        df_verif.at[index, 'Plates_1'] = np.nan
    elif np.any(v2 == False):
        df_verif.at[index, 'Plates_2'] = np.nan
pprint(df_verif, set_pandas_options(df_verif, width=1000, colwidth=30, colmap = False)) 

     Plates_1                      Verif_1   Plates_2                       Verif_2
0   О123ЕХ777     [True, True, True, True]        NaN  [False, False, False, False]
1   О456ЕР199     [True, True, True, True]        NaN  [False, False, False, False]
2   С789НХ797     [True, True, True, True]        NaN   [False, False, False, True]
3   К321РО177     [True, True, True, True]        NaN   [False, False, False, True]
4   У654ОУ799     [True, True, True, True]        NaN  [False, False, False, False]
5   О987РЕ797     [True, True, True, True]        NaN   [False, False, False, True]
6         NaN  [False, False, False, True]  Е456СР799      [True, True, True, True]
7         NaN  [False, False, False, True]  Р123РХ799      [True, True, True, True]
8   О197СС777     [True, True, True, True]        NaN   [False, False, False, True]
9   У799СС799     [True, True, True, True]        NaN  [False, False, False, False]
10  Р199ЕХ197     [True, True, True, True]        NaN   [False, False, False

In [11]:
# Создаем базовый датафрейм с действительными гос.номерами
df_base = df_verif.loc[:, ['Plates_1', 'Plates_2']]
pprint(df_base)

     Plates_1   Plates_2
0   О123ЕХ777        NaN
1   О456ЕР199        NaN
2   С789НХ797        NaN
3   К321РО177        NaN
4   У654ОУ799        NaN
5   О987РЕ797        NaN
6         NaN  Е456СР799
7         NaN  Р123РХ799
8   О197СС777        NaN
9   У799СС799        NaN
10  Р199ЕХ197        NaN
11  У977ЕС199        NaN
12  О123УХ777        NaN
13  Р799СЕ799        NaN
14  У797УХ777        NaN
15        NaN  О777ОР799
16  Е799СС797        NaN
17  Р197ОС199        NaN
18  К199НС777        NaN
19  О123УХ799        NaN
20  С456НУ799        NaN
21  О789РО799        NaN
22        NaN  С234ОР799
23  У512ОУ777        NaN
24  К888ЕХ799        NaN
25  У798ОН799        NaN
26  К111УО799        NaN
27        NaN  О123ЕР799
28  Р234ОС799        NaN
29  У512УУ799        NaN


In [12]:
# Сливаем 2 колонки по принципу "Валидный гос.номер остается, нулевое значение уходит"
for index, (p1, p2) in df_base[['Plates_1', 'Plates_2']].iterrows():
        if not pd.isna(p2):
            df_base.at[index, 'Plates_1'] = p2
            df_base.at[index, 'Plates_2'] = np.nan

L_plates = df_base['Plates_1'].tolist()

# Обновляем исходный датафрейм
df['Plates'] = L_plates

pprint(df, set_pandas_options(df, width=1000, colwidth=20, colmap = False)) 

           Brands           Locations                Units     Plates              Drivers Ratings
0       Chevrolet      Центральный АО     Chevrolet Camaro  О123ЕХ777  Петров Александр...     4.5
1       Chevrolet      Центральный АО     Chevrolet Impala  О456ЕР199  Смирнова Елена Н...     4.2
2       Chevrolet      Центральный АО     Chevrolet Malibu  С789НХ797  Кузнецов Сергей ...     4.8
3       Chevrolet      Центральный АО      Chevrolet Tahoe  К321РО177  Васильева Анна А...     3.9
4       Chevrolet      Центральный АО    Chevrolet Equinox  У654ОУ799  Павлова Ольга Ми...     4.6
5       Chevrolet      Центральный АО   Chevrolet Traverse  О987РЕ797  Михайлов Игорь В...     4.3
6       Chevrolet      Центральный АО  Chevrolet Silverado  Е456СР799  Иванов Ирина Сер...     4.9
7            Ford         Северный АО         Ford Mustang  Р123РХ799  Иванов Александр...     4.8
8            Ford         Северный АО           Ford F-150  О197СС777  Смирнов Александ...     4.6
9         